---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevincrotty.rice.edu/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [10]:
# stocks/funds to study
tickers = (
    "SPY",
    "GLD",
    "IEF",
    "LQD"
)

# risk-free rate (monthly)
rf = 0.02 / 12

# or savings rate and borrowing rate (monthly)
rs = 0.02 / 12
rb = 0.05 / 12

# risk aversion
raver = 6

# GET DATA

In [3]:
import yfinance as yf

df = yf.download(" ".join(tickers), start="1970-01-01")
df = df["Adj Close"].resample("M").last().pct_change().dropna().iloc[:-1]

[*********************100%***********************]  4 of 4 completed


# COMPUTE HISTORICAL PARAMETERS

In [11]:
mn = df.mean()
sd = df.std()
cov = df.cov()

num_assets = df.shape[1]

# INSTALL LIBRARIES

In [12]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# OPTIMAL PORTFOLIO 1

Use this if the borrowing and savings rate are the same and short sales are allowed.

The mean and standard deviation are annualized.

In [13]:
import numpy as np
import pandas as pd

optimum = np.linalg.solve(raver*cov, mn-rf)
optimum_mn = 12 * (rf + optimum @ (mn-rf))
optimum_sd = np.sqrt(12 * optimum @ cov @ optimum)

print(f"optimal savings is {max(0, 1-np.sum(optimum)): 0.3f}")
print(f"optimal borrowing is {max(0, np.sum(optimum)-1): 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

optimal savings is  0.000
optimal borrowing is  0.341
optimal expected return is  0.101
optimal standard deviation is  0.116
optimal portfolio is


GLD    0.293101
IEF    1.153926
LQD   -0.927969
SPY    0.822367
dtype: float64

# OPTIMAL PORTFOLIO 2

Use this if the borrowing rate is greater than the savings rate and short sales are allowed.    

The mean and standard deviation are annualized.

In [14]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0
G = np.zeros((2, num_assets+2))
G[0, 0] = G[1, 1] = -1
h = np.zeros(2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (2, num_assets+2))
h = matrix(h, (2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = 12 * (xs*rs - xb*rb + optimum @ mn)
optimum_sd = np.sqrt(12 * optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

     pcost       dcost       gap    pres   dres
 0: -4.9581e-03 -4.9540e-03  3e-03  1e+00  2e-18
 1: -4.9540e-03 -4.9540e-03  3e-05  1e-02  1e-18
 2: -4.9540e-03 -4.9540e-03  3e-07  1e-04  1e-18
 3: -4.9540e-03 -4.9540e-03  3e-09  1e-06  2e-18
 4: -4.9540e-03 -4.9540e-03  3e-11  1e-08  2e-18
Optimal solution found.
optimal savings is  0.000
optimal borrowing is -0.000
optimal expected return is  0.092
optimal standard deviation is  0.105
optimal portfolio is


GLD    0.295431
IEF    0.843902
LQD   -0.887948
SPY    0.748615
dtype: float64

# OPTIMAL PORTFOLIO 3

Use this if short sales are not allowed.  If the borrowing and savings rate are the same (=rf) then add the line:

    rb = rs = rf

The mean and standard deviation are annualized.

In [15]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0 and -w <= 0
G = -np.identity(num_assets+2)
h = np.zeros(num_assets+2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (num_assets+2, num_assets+2))
h = matrix(h, (num_assets+2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = 12 * (xs*rs - xb*rb + optimum @ mn)
optimum_sd = np.sqrt(12 * optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal std dev is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

     pcost       dcost       gap    pres   dres
 0: -3.8682e-03 -6.1445e-01  8e+00  3e+00  3e+00
 1:  8.5981e-04 -1.2910e+00  2e+00  6e-01  6e-01
 2:  8.0955e-03 -5.6037e-02  6e-02  2e-02  2e-02
 3:  6.4904e-04 -7.2342e-03  8e-03  2e-16  3e-18
 4: -3.7638e-03 -4.9557e-03  1e-03  3e-16  1e-18
 5: -4.4549e-03 -4.5142e-03  6e-05  6e-17  9e-19
 6: -4.4840e-03 -4.4852e-03  1e-06  2e-16  8e-19
 7: -4.4848e-03 -4.4848e-03  1e-08  1e-16  1e-18
Optimal solution found.
optimal savings is  0.000
optimal borrowing is  0.000
optimal expected return is  0.083
optimal std dev is  0.098
optimal portfolio is


GLD    0.283148
IEF    0.189722
LQD    0.000006
SPY    0.527123
dtype: float64